In [8]:
import sqlite3, json, random
from datetime import datetime, timedelta
from tqdm.notebook import tqdm, trange

con = sqlite3.connect('test.db')
cur = con.cursor()

cur.execute('DROP TABLE IF EXISTS StockDB')
cur.execute('CREATE TABLE StockDB \
    (stock_id integer PRIMARY KEY, \
    name text, \
    data text)')

cur.execute('DROP TABLE IF EXISTS IssueDB')
cur.execute('CREATE TABLE IssueDB \
    (issue_id integer PRIMARY KEY, \
    issue text, \
    data text)')

cur.execute('DROP TABLE IF EXISTS NewsDB')
cur.execute('CREATE TABLE NewsDB \
    (news_id integer PRIMARY KEY, \
    data text)')

con.close()

In [9]:
con = sqlite3.connect('test.db')
cur = con.cursor()

n_stocks = 10
n_issues = n_stocks*10
issue_pop = [i for i in range(n_issues)]
n_news = n_issues*20
news_pop = [i for i in range(n_news)]

# write stock_db data 
for i in trange(n_stocks, desc='write stock_db data'):
    name = 'stock_name_{:03d}'.format(i)
    data = {
        'stock_id': i,
        'name': name,
        'issues': sorted(random.sample(issue_pop, random.randrange(1,10))),
    }
    
    query = 'INSERT INTO StockDB VALUES (:stock_id, :name, :data)'
    parameters = {
        'stock_id': i,
        'name': name,
        'data': json.dumps(data)
    }
    cur.execute(query, parameters)

# write issue_db data 
for i in trange(n_issues, desc='write issue_db data'):
    issue = 'this is an issue for issue_id={:03d}'.format(i)
    date_i = datetime(random.randrange(2019,2021),random.randrange(1,13),random.randrange(1,29))
    date_f = date_i + timedelta(days=random.randrange(10,31))
    data = {
        'issue_id': i,
        'issue': issue,
        'date_i': date_i.strftime('%Y-%m-%d'),
        'date_f': date_f.strftime('%Y-%m-%d'),
        'news': sorted(random.sample(news_pop, random.randrange(1,n_news//n_issues*2+1))),
    }
    
    query = 'INSERT INTO IssueDB VALUES (:issue_id, :issue, :data)'
    parameters = {
        'issue_id': i,
        'issue': issue,
        'data': json.dumps(data)
    }
    cur.execute(query, parameters)

# write news_db data 
for i in trange(n_news, desc='write news_db data'):
    date = datetime(random.randrange(2019,2021),random.randrange(1,13),random.randrange(1,29))
    title = 'news_title_{:03d}'.format(i)
    data = {
        'news_id': i,
        'title': title,
        'date': date_i.strftime('%Y-%m-%d'),
        'text':  'this is a news for news_id={:03d}'.format(i),
        'url': 'https://www.url.com/{:03d}'.format(i)
    }
    
    query = 'INSERT INTO NewsDB VALUES (:news_id, :data)'
    parameters = {
        'news_id': i,
        'data': json.dumps(data)
    }
    cur.execute(query, parameters)
    
con.commit()

# read data and convert string to dictionary in db
cur.execute('SELECT * FROM StockDB')
stock_db = cur.fetchall()
for i, element in enumerate(tqdm(stock_db, desc='convert str to dict')):
    stock_db[i] = list(element)
    stock_db[i][2] = json.loads(element[2])

cur.execute('SELECT * FROM IssueDB')
issue_db = cur.fetchall()
for i, element in enumerate(tqdm(issue_db, desc='convert str to dict')):
    issue_db[i] = list(element)
    issue_db[i][2] = json.loads(element[2])

cur.execute('SELECT * FROM NewsDB')
news_db = cur.fetchall()
for i, element in enumerate(tqdm(news_db, desc='convert str to dict')):
    news_db[i] = list(element)
    news_db[i][1] = json.loads(element[1])

# write issue_db and news_db to .json file
with open('./stock_db.json', 'w') as json_file:
    json.dump(stock_db, json_file, indent=4)
with open('./issue_db.json', 'w') as json_file:
    json.dump(issue_db, json_file, indent=4)
with open('./news_db.json', 'w') as json_file:
    json.dump(news_db, json_file, indent=4)
    
con.close()

write stock_db data:   0%|          | 0/10 [00:00<?, ?it/s]

write issue_db data:   0%|          | 0/100 [00:00<?, ?it/s]

write news_db data:   0%|          | 0/2000 [00:00<?, ?it/s]

convert str to dict:   0%|          | 0/10 [00:00<?, ?it/s]

convert str to dict:   0%|          | 0/100 [00:00<?, ?it/s]

convert str to dict:   0%|          | 0/2000 [00:00<?, ?it/s]